# Eager Execution

TensorFlow의 Eager excution은 추가적인 그래프 작성 단계없이 작업을 즉시 평가하는 필수 프로그래밍 환경입니다. Operations는 나중에 실행할 계산 그래프를 작성하는 대신 구체적인 값을 리턴합니다. 따라서 TensorFlow를 시작하고, 모델을 디버깅하고, boilerplate code를 줄이고, 또한 재미가 있습니다! 이 가이드를 따르려면 대화 형 파이썬 인터프리터에서 아래 코드 샘플을 실행하십시오.

Eager excution은 대부분의 TensorFlow 작업과 GPU 가속을 지원합니다. Automatic differentiation은 그라디언트를 계산하기 위해 정적 그래프 대신 동적으로 구성된 테이프를 사용합니다. Eager excution은 다음을 제공하는 연구 및 실험을 위한 유연한 기계 학습 플랫폼입니다.

* 직관적 인 인터페이스 - 자연스럽게 코드를 구성하고 파이썬 데이터 구조를 사용합니다. 작은 모델과 작은 데이터를 빠르게 반복 할 수 있습니다.
* 더 쉬운 디버깅 - 실행중인 모델을 검사하고 변경 사항을 테스트하기 위해 직접 ops를 호출합니다. 즉각적인 오류보고를 위해 표준 Python 디버깅 도구를 사용하십시오.
* 자연 제어 흐름 - 동적 모델 지원을 포함하여 그래프 제어 흐름 대신 파이썬 제어 흐름을 사용합니다.

eager 실행으로 실행되는 예제 모음은 [tensorflow/contrib/eager/python/examples](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples)를 참조하십시오.

Note : 일부 모델의 경우 열심히 실행하면 오버 헤드가 증가 할 수 있습니다. 성능 향상은 계속되고 있지만, 문제를 발견하고 벤치 마크를 공유하면 버그를 신고하십시오.

## Setup and basic usage

### Upgrade to TensorFlow 1.7 to include updates for eager execution:

In [4]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: astor>=0.6.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: tensorboard<1.8.0,>=1.7.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: numpy>=1.13.3 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: six>=1.10.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: gast>=0.2.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: protobuf>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: absl-py>=0.1.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: wheel>=0.26 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: termcolor>=1.1.0 in /usr/lo

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

#### 기존 버전 ( <= 1.6)

In [2]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))  # => "hello, [[4.]]"

hello, Tensor("MatMul:0", shape=(1, 1), dtype=float32)


#### tf.enable_eager_execution() 적용한 버전

In [2]:
tf.enable_eager_execution()

x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))  # => "hello, [[4.]]"

hello, [[4.]]


In [4]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)
# => tf.Tensor([[1 2]
#               [3 4]], shape=(2, 2), dtype=int32)

# Broadcasting support
b = tf.add(a, 1)
print(b)
# => tf.Tensor([[2 3]
#               [4 5]], shape=(2, 2), dtype=int32)

# Operator overloading is supported
print(a * b)
# => tf.Tensor([[ 2  6]
#               [12 20]], shape=(2, 2), dtype=int32)

print("\nUse NumPy values")

# Use NumPy values
import numpy as np

c = np.multiply(a, b)
print(c)
# => [[ 2  6]
#     [12 20]]

# Obtain numpy value from a tensor:
print(a.numpy())
# => [[1 2]
#     [3 4]]

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)

Use NumPy values
[[ 2  6]
 [12 20]]
[[1 2]
 [3 4]]


tfe 모듈에는 eager 및 그래프 실행 환경에서 사용할 수있는 심볼이 들어 있으며 그래프 작업을 위한 코드를 작성하는 데 유용합니다.

In [3]:
import tensorflow.contrib.eager as tfe

Instructions for updating:
Use the retry module or similar alternatives.


### Eager training

#### Automatic differentiation

Automatic differentiation은 신경 네트워크 교육을 위한 역 전파와 같은 기계 학습 알고리즘을 구현하는 데 유용합니다. 열심히 실행하는 동안 tfe.GradientTape를 사용하여 나중에 그라디언트를 계산하기위한 작업을 추적하십시오.

tfe.GradientTape는 추적하지 않을 때 최대한의 성능을 제공하는 opt-in 기능입니다. 각 호출 중에 다른 작업이 발생할 수 있으므로 모든 순방향 전달 작업이 "테이프"에 기록됩니다. 그래디언트를 계산하려면 테이프를 뒤로 움직 인 다음 폐기하십시오. 특정 tfe.GradientTape는 한 번만 계산할 수 있으며 후속 호출은 런타임 오류를 발생시킵니다.

In [7]:
w = tfe.Variable([[1.0]])
with tfe.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, [w])
print(grad)  # => [tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)]

[<tf.Tensor: id=49, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


Here's an example of tfe.GradientTape that records forward-pass operations to train a simple model:

In [8]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
  return input * weight + bias

# A loss function using mean-squared error
def loss(weights, biases):
  error = prediction(training_inputs, weights, biases) - training_outputs
  return tf.reduce_mean(tf.square(error))

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
  with tfe.GradientTape() as tape:
    loss_value = loss(weights, biases) 
  return tape.gradient(loss_value, [weights, biases])

train_steps = 200
learning_rate = 0.01
# Start with arbitrary values for W and B on the same batch of data
W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial loss: {:.3f}".format(loss(W, B)))

for i in range(train_steps):
  dW, dB = grad(W, B)
  W.assign_sub(dW * learning_rate)
  B.assign_sub(dB * learning_rate)
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final loss: {:.3f}".format(loss(W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))

Initial loss: 68.481
Loss at step 000: 65.824
Loss at step 020: 30.025
Loss at step 040: 13.998
Loss at step 060: 6.821
Loss at step 080: 3.608
Loss at step 100: 2.169
Loss at step 120: 1.524
Loss at step 140: 1.235
Loss at step 160: 1.106
Loss at step 180: 1.048
Final loss: 1.023
W = 3.079099416732788, B = 2.141737699508667


Replay the tfe.GradientTape to compute the gradients and apply them in a training loop.

#### Dynamic models

tfe.GradientTape는 동적 모델에서도 사용할 수 있습니다. backtracking line search 알고리즘에 대한이 예제는 복잡한 제어 흐름에도 불구하고 gradients가 있고 미분 가능하다는 점을 제외하고는 일반적인 NumPy 코드와 유사합니다.

In [0]:
def line_search_step(fn, init_x, rate=1.0):
  with tfe.GradientTape() as tape:
    # Variables are automatically recorded, but manually watch a tensor
    tape.watch(init_x)
    value = fn(init_x)
  grad, = tape.gradient(value, [init_x])
  grad_norm = tf.reduce_sum(grad * grad)
  init_value = value
  while value > init_value - rate * grad_norm:
    x = init_x - rate * grad
    value = fn(x)
    rate /= 2.0
  return x, value

#### Additional functions to compute gradients

tfe.GradientTape는 gradients를 계산하기 위한 강력한 인터페이스이지만 automatic differentiation을 위해 사용할 수있는 또 다른 Autograd-style API가 있습니다. 이 함수는 텐서 및 gradient 함수 만 사용하고 tfe가 없는 수학 코드를 작성하는 경우에 유용합니다. 변수 :

* tfe.gradients_function - 인수와 관련하여 입력 함수 매개 변수의 미분을 계산하는 함수를 반환합니다. 입력 함수 매개 변수는 스칼라 값을 반환해야합니다. 반환 된 함수가 호출되면 tf.Tensor 객체 목록을 반환합니다. 입력 함수의 각 인수에 대해 하나의 요소입니다. 관심있는 것이 무엇이든 함수 매개 변수로 전달되어야하므로, 많은 훈련 가능한 매개 변수에 의존성이 있으면 다루기 힘들어집니다.
* tfe.value_and_gradients_function - tfe.gradients_function과 비슷하지만 리턴 된 함수가 호출 될 때 입력 함수의 인수와 관련하여 입력 함수의 파생어 목록 이외에 입력 함수의 값을 리턴합니다.

다음 예제에서 tfe.gradients_function은 square 함수를 인수로 사용하고 입력에 대해 square의 편미분도를 계산하는 함수를 반환합니다. 숫자 3의 square에 대한 미분을 계산하려면 grad (3.0)이 6을 반환합니다.

In [6]:
def square(x):
  return tf.multiply(x, x)

grad = tfe.gradients_function(square)

print(square(3.))  # => 9.0
print(grad(3.))    # => [6.0]

tf.Tensor(9.0, shape=(), dtype=float32)
[<tf.Tensor: id=58, shape=(), dtype=float32, numpy=6.0>]


In [7]:
# The second-order derivative of square:
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])
gradgrad(3.)  # => [2.0]

[<tf.Tensor: id=70, shape=(), dtype=float32, numpy=2.0>]

In [8]:
# The third-order derivative is None:
gradgradgrad = tfe.gradients_function(lambda x: gradgrad(x)[0])
gradgradgrad(3.)  # => [None]

[None]

In [9]:
# With flow control:
def abs(x):
  return x if x > 0. else -x

grad = tfe.gradients_function(abs)

print(grad(3.))   # => [1.0]
print(grad(-3.))  # => [-1.0]

[<tf.Tensor: id=5, shape=(), dtype=float32, numpy=1.0>]
[<tf.Tensor: id=94, shape=(), dtype=float32, numpy=-1.0>]


#### Custom gradients

### Build and train models

Eager execution은 tf.keras.layers 모듈에서 Keras 스타일 레이어 클래스의 사용을 권장합니다. 또한 tf.train.Optimizer 클래스는 매개 변수 업데이트를 계산하는 정교한 기술을 제공합니다.

#### Build a model

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(784,)),  # must declare input shape
  tf.keras.layers.Dense(10)
])

In [0]:
class MNISTModel(tf.keras.Model):
  def __init__(self):
    super(MNISTModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(units=10)
    self.dense2 = tf.keras.layers.Dense(units=10)

  def call(self, input):
    """Run the model."""
    result = self.dense1(input)
    result = self.dense2(result)
    result = self.dense2(result)  # reuse variables from dense2 layer
    return result

model = MNISTModel()

#### Train a model

Even without training, call the model and inspect the output in eager execution:

In [12]:
# Create a tensor representing a blank image
batch = tf.zeros([1, 1, 784])
print(batch.shape)  # => (1, 1, 784)

result = model(batch)
print(result) # => tf.Tensor([[[ 0.  0., ..., 0.]]], shape=(1, 1, 10), dtype=float32)

(1, 1, 784)
tf.Tensor([[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]], shape=(1, 1, 10), dtype=float32)


#### Variables and optimizers

tfe.Variable는 automatic differentiation를 쉽게하기 위해 학습 중에 액세스 할 수있는 변경 가능한 tf.Tensor 값을 저장합니다. 모델의 매개 변수는 클래스로 변수로 캡슐화 될 수 있습니다.

tfe.GradientTape와 함께 tfe.Variable을 사용하여 모델 매개 변수를보다 잘 캡슐화합니다. 예를 들어 위의 automatic differentiation 예제를 다시 작성할 수 있습니다.

In [24]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tfe.Variable(5., name='weight')
    self.B = tfe.Variable(10., name='bias')
  def predict(self, inputs):
    return inputs * self.W + self.B

# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 2000
training_inputs = tf.random_normal([NUM_EXAMPLES])
print("inputs : {}".format(training_inputs)) # print inputs
noise = tf.random_normal([NUM_EXAMPLES])
print("noise : {}".format(noise)) # print noise
training_outputs = training_inputs * 3 + 2 + noise

# The loss function to be optimized
def loss(model, inputs, targets):
  error = model.predict(inputs) - targets
  return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
  with tfe.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

# Define:
# 1. A model.
# 2. Derivatives of a loss function with respect to model parameters.
# 3. A strategy for updating the variables based on the derivatives.
model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# Training loop
for i in range(300):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                            global_step=tf.train.get_or_create_global_step())
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

inputs : [-0.80618113  0.0305943   0.26421696 ...  0.94597554  0.20125063
  2.0738869 ]
noise : [ 0.930609    0.07488708  1.4669206  ...  0.2983844  -2.2759821
  0.91414446]
Initial loss: 70.024
Loss at step 000: 67.239
Loss at step 020: 30.071
Loss at step 040: 13.754
Loss at step 060: 6.586
Loss at step 080: 3.436
Loss at step 100: 2.050
Loss at step 120: 1.440
Loss at step 140: 1.171
Loss at step 160: 1.052
Loss at step 180: 1.000
Loss at step 200: 0.977
Loss at step 220: 0.967
Loss at step 240: 0.962
Loss at step 260: 0.960
Loss at step 280: 0.959
Final loss: 0.959
W = 2.9771499633789062, B = 2.0310218334198


### Use objects for state during eager execution

graph execution과 함께, 프로그램 상태 (예 : Variable)는 전역 콜렉션에 저장되며 수명은 tf.Session 오브젝트에 의해 관리됩니다. 반대로, eager execution 동안, 상태 객체의 수명은 해당 Python 객체의 수명에 따라 결정됩니다.

#### Variables are objects

eager execution 동안 변수는 객체에 대한 마지막 참조가 제거 될 때까지 지속 된 다음 삭제됩니다.

In [0]:
with tf.device("CPU:0"):
  v = tfe.Variable(tf.random_normal([1000, 1000]))
  v = None  # v no longer takes up GPU memory

#### Object-based saving

tfe.Checkpoint은 체크 포인트와의 사이에 tfe.Variables를 저장하고 복원 할 수 있습니다.

In [33]:
x = tfe.Variable(10.)

checkpoint = tfe.Checkpoint(x=x)  # save as "x"

x.assign(2.)   # Assign a new value to the variables and save.
save_path = checkpoint.save('./ckpt/')

x.assign(11.)  # Change the variable after saving.

# Restore values from the checkpoint
checkpoint.restore(save_path)

print(x)  # => 2.0

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


모델을 저장하고 로드하기 위해 tfe.Checkpoint는 hidden variables를 필요로 하지 않고 객체의 내부 상태를 저장합니다. 모델, 최적화 프로그램 및 전역 단계의 상태를 기록하려면 tfe.Checkpoint에 해당 단계를 전달하십시오.

In [36]:
model = Model()
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
checkpoint_dir = './ckpt/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tfe.Checkpoint(optimizer=optimizer,
                      model=model,
                      optimizer_step=tf.train.get_or_create_global_step())

root.save(file_prefix=checkpoint_prefix)
# or
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

#### Object-oriented metrics

tfe.metrics는 객체로 저장됩니다. callable에 새 데이터를 전달하여 메트릭을 업데이트하고 tfe.metrics.result 메소드를 사용하여 결과를 검색하십시오. 예를 들면 다음과 같습니다.

In [37]:
m = tfe.metrics.Mean("loss")
m(0)
m(5)
print("m.result() : {}".format(m.result()))  # => 2.5
m([8, 9])
print("m.result() : {}".format(m.result()))  # => 5.5

m.result() : 2.5
m.result() : 5.5


#### Summaries and TensorBoard

TensorBoard는 모델 교육 과정을 이해하고, 디버깅하고 최적화하기위한 시각화 도구입니다. 프로그램을 실행하는 동안 작성된 요약 이벤트를 사용합니다.

tf.contrib.summary는 eager 및 그래프 실행 환경과 호환됩니다. 모델 구성 중에 tf.contrib.summary.scalar와 같은 요약 작업이 삽입됩니다. 예를 들어, 100 개의 글로벌 단계마다 요약을 기록하려면 다음과 같이하십시오.

In [0]:
writer = tf.contrib.summary.create_file_writer(logdir)
global_step=tf.train.get_or_create_global_step()  # return global step var

writer.set_as_default()

for _ in range(iterations):
  global_step.assign_add(1)
  # Must include a record_summaries method
  with tf.contrib.summary.record_summaries_every_n_global_steps(100):
    # your model code goes here
    tf.contrib.summary.scalar('loss', loss)
     ...

### Performance

#### Benchmarks

### Work with graphs

eager execution으로 개발 및 디버깅을보다 상호 작용 적으로 수행 할 수 있지만 TensorFlow 그래프 실행은 분산 된 교육, 성능 최적화 및 프로덕션 배포에 이점이 있습니다. 그러나 그래프 코드를 작성하는 것은 일반적인 파이썬 코드 작성과 디버깅하기가 더 어려울 수 있습니다.

graph-constructed models을 구축하고 교육하기 위해 Python 프로그램은 먼저 계산을 나타내는 그래프를 작성한 다음 Session.run을 호출하여 C ++ 기반 런타임에서 실행을 위해 그래프를 보냅니다. 이것은 다음을 제공합니다.

* static autodiff를 사용한 Automatic differentiation.
* 플랫폼 독립적 서버에 대한 간단한 배포.
* 그래프 기반 최적화 (공통 하위 표현식 제거, constant-folding 등)
* Compilation 및 kernel fusion
* 자동 배포 및 복제 (분산 시스템에 노드 배치).

 eager execution을 위해 작성된 코드를 배포하는 것은 더 어렵습니다. 모델에서 그래프를 생성하거나 Python 런타임 및 코드를 서버에서 직접 실행하십시오.

#### Write compatible code

eager execution을 위해 작성된 동일한 코드는 그래프 실행 도중 그래프를 작성합니다. eager execution이 활성화되지 않은 새로운 Python 세션에서 동일한 코드를 실행하면됩니다.

대부분의 TensorFlow 작업은 eager execution 동안 작동하지만 몇 가지 사항을 염두에 두어야합니다.

* queues 대신 입력 처리를 위해 tf.data를 사용하십시오. 빠르고 쉽습니다.
* 변수에 대한 명시적인 저장소가 있으므로, object-oriented layer APIs를 사용하십시오-tf.keras.layers 및 tf.keras.Model과 같은
* 대부분의 모델 코드는 eager execution과 그래프를 실행하는 동안 동일하게 작동하지만 예외가 있습니다. (예를 들어, 입력을 기반으로 계산을 변경하기 위해 Python 제어 흐름을 사용하는 동적 모델).
* tf.enable_eager_execution으로 eager 실행을 활성화 한 후에는 해제 할 수 없습니다. 새로운 Python 세션을 시작하여 그래프 실행으로 돌아갑니다.

eager execution과 그래프 실행을 위한 코드를 작성하는 것이 가장 좋습니다. 따라서 그래프 실행의 분산 된 성능 이점을 통해 대화식 실험 및 디버깅 기능을 제공합니다.

코드의 write, debug, iterate를 eager execution에서 작성하고, 프로덕션 배포를 위해 모델 그래프를 가져옵니다. tfe.Checkpoint를 사용하여 모델 변수를 저장하고 복원하면 eager execution과 그래프 실행 환경 사이를 이동할 수 있습니다. 예 : tensorflow / contrib / eager / python / examples을 참조하십시오.

#### Use eager execution in a graph environment

tfe.py_func를 사용하여 TensorFlow 그래프 환경에서 eager execution을 선택적으로 활성화합니다. 이것은 tf.enable_eager_execution ()이 호출되지 않았을 때 사용됩니다

##### eager excution enable 중 일때 에러메시지

In [38]:
def my_py_func(x):
  x = tf.matmul(x, x)  # You can use tf ops
  print(x)  # but it's eager!
  return x

with tf.Session() as sess:
  x = tf.placeholder(dtype=tf.float32)
  # Call eager function in graph!
  pf = tfe.py_func(my_py_func, [x], tf.float32)
  sess.run(pf, feed_dict={x: [[2.0]]})  # [[4.0]]

RuntimeError: ignored

##### eager execution 해제 후 실행

In [5]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

def my_py_func(x):
  x = tf.matmul(x, x)  # You can use tf ops
  print(x)  # but it's eager!
  return x

with tf.Session() as sess:
  x = tf.placeholder(dtype=tf.float32)
  # Call eager function in graph!
  pf = tfe.py_func(my_py_func, [x], tf.float32)
  sess.run(pf, feed_dict={x: [[2.0]]})  # [[4.0]]

tf.Tensor([[4.]], shape=(1, 1), dtype=float32)
